# 2 Mean-Variance Optimization

- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill
fund return, (“SHV”,) which we consider as the risk-free rate.
- The data is provided in total returns, (in which case you should ignore the SHV column,) as
well as excess returns, (where SHV has been subtracted from the other columns.)
- These are nominal returns—they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.



In [322]:
import pandas as pd
import numpy as np
import math as m

In [323]:
ret = pd.read_excel('multi_asset_etf_data.xlsx',2)
ret.set_index('Date', inplace=True)
del ret['SHV']
ret

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008993,-0.001000,0.155582,0.115190,0.138460,-0.027452,0.296151,0.230202,0.022882,0.099346,-0.017952
2009-05-31,0.053672,0.162663,0.159400,0.131918,0.028555,-0.020773,0.022727,0.053892,0.027865,0.058454,0.019967
2009-06-30,0.005149,-0.026259,-0.022495,-0.014050,0.033516,-0.005572,-0.024863,0.045449,-0.003436,-0.000655,0.001982
2009-07-31,0.031284,0.018568,0.110146,0.100415,0.069191,0.008317,0.105799,0.143247,0.015326,0.074606,0.000879
2009-08-31,0.007628,-0.040365,-0.013136,0.045031,-0.016969,0.007635,0.131939,0.033413,-0.004151,0.036939,0.008413
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30,-0.069696,0.056408,-0.061351,-0.067391,-0.041803,-0.042283,-0.041305,-0.125679,-0.033398,-0.087769,-0.021831
2022-05-31,0.005460,0.046131,0.006135,0.019959,0.016299,0.006184,-0.044434,0.015084,-0.004025,0.002257,-0.009922
2022-06-30,-0.046443,-0.075000,-0.051577,-0.087666,-0.070499,-0.008634,-0.068911,-0.132477,-0.033681,-0.082460,-0.031155


In the questions below, annualize the statistics you report.
- Annualize the mean of monthly returns with a scaling of 12.

In [324]:
ret_mean = ret.mean()*12
ret_mean.head(7)

BWX    0.004653
DBC    0.038846
EEM    0.072621
EFA    0.081124
HYG    0.071588
IEF    0.025833
IYR    0.150128
dtype: float64

- Annualize the volatility of monthly returns with a scaling of $\sqrt12$

In [325]:
ret_vol = ret.std()*m.sqrt(12)
ret_vol.head(7)

BWX    0.078535
DBC    0.180186
EEM    0.191787
EFA    0.161885
HYG    0.089403
IEF    0.060077
IYR    0.184407
dtype: float64

- The Sharpe Ratio is the mean return divided by the volatility of returns.3 Accordingly, we can
annualize the Sharpe Ratio with a scaling of sqrt(12)
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe).

In [326]:
ret_sp_ratio = ret_mean/ret_vol
ret_sp_ratio.head(7)

BWX    0.059248
DBC    0.215590
EEM    0.378655
EFA    0.501125
HYG    0.800730
IEF    0.429996
IYR    0.814113
dtype: float64

In [327]:
table = pd.concat([ret_mean, ret_vol, ret_sp_ratio], axis=1)
table = table.rename(columns={0:'Mean', 1:'Vol', 2:'Sharpe Ratio'})
table

,Mean,Vol,Sharpe Ratio
BWX,0.004653,0.078535,0.059248
DBC,0.038846,0.180186,0.215590
EEM,0.072621,0.191787,0.378655
EFA,0.081124,0.161885,0.501125
HYG,0.071588,0.089403,0.800730
IEF,0.025833,0.060077,0.429996
IYR,0.150128,0.184407,0.814113
PSP,0.133272,0.221299,0.602227
QAI,0.022862,0.048879,0.467723
SPY,0.150293,0.144811,1.037857


### We are going to analyze the problem in terms of total–not excess–returns.
- Thus, you will focus on the “Mean-Variance” section of the lecture notes, especially the formulas on slide 40.
- In using the “total returns” tab of the data, drop the column SHV. It is our proxy for the risk-free rate, which we are ignoring in our analysis of total returns.
- Thus, below, you are analyzing 11 risky assets–not SHV.

## 1. Summary Statistics
- Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation
- Which assets have the best and worst Sharpe ratios4?

## 1.(a) EXCESS RETURN TABLE

In [328]:
ex_ret = pd.read_excel('multi_asset_etf_data.xlsx',3)
ex_ret.set_index('Date', inplace=True)
table2 = pd.concat([ex_ret.mean()*12, ex_ret.std()*m.sqrt(12), ex_ret.mean()*12/(ex_ret.std()*m.sqrt(12))], axis=1)
table2 = table2.rename(columns={0:'Mean', 1:'Vol', 2:'Sharpe Ratio'})
table2

,Mean,Vol,Sharpe Ratio
BWX,0.000003,0.078307,0.000034
DBC,0.034196,0.180663,0.189279
EEM,0.067971,0.192071,0.353884
EFA,0.076474,0.162298,0.471197
HYG,0.066938,0.089701,0.746233
IEF,0.021182,0.059387,0.356685
IYR,0.145477,0.184744,0.787452
PSP,0.128622,0.221773,0.579971
QAI,0.018212,0.049174,0.370346
SPY,0.145643,0.145260,1.002640


## 1.(b) BEST AND WORST SHARPE RATIOS

In [329]:
#best and worst sharpe ratio

print('Best Sharpe Ratio: ', table['Sharpe Ratio'].idxmax())
print('Worst Sharpe Ratio: ', table['Sharpe Ratio'].idxmin())

Best Sharpe Ratio:  SPY
Worst Sharpe Ratio:  BWX


In [330]:
table.nlargest(1, 'Sharpe Ratio')

,Mean,Vol,Sharpe Ratio
SPY,0.150293,0.144811,1.037857


In [331]:
table.nsmallest(1, 'Sharpe Ratio')

,Mean,Vol,Sharpe Ratio
BWX,0.004653,0.078535,0.059248


## 2. Descriptive Analysis


### 2.(a) Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?

In [332]:
corr = ret.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(4)

/var/folders/js/d6m3lmg13x1dqjyw_1h8pr8h0000gn/T/ipykernel_22591/1148366645.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='coolwarm').set_precision(4)


,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
BWX,1.0000,0.3276,0.6032,0.5758,0.5375,0.3515,0.4116,0.5087,0.6412,0.4272,0.5770
DBC,0.3276,1.0000,0.5646,0.5774,0.4507,-0.4054,0.2842,0.4838,0.5305,0.4909,0.0732
EEM,0.6032,0.5646,1.0000,0.8465,0.7247,-0.2376,0.5988,0.7865,0.7924,0.7346,0.2297
EFA,0.5758,0.5774,0.8465,1.0000,0.7628,-0.2648,0.6809,0.9087,0.8370,0.8740,0.2001
HYG,0.5375,0.4507,0.7247,0.7628,1.0000,-0.0888,0.7470,0.8240,0.7608,0.7576,0.3086
IEF,0.3515,-0.4054,-0.2376,-0.2648,-0.0888,1.0000,-0.0186,-0.2459,-0.0400,-0.2692,0.6803
IYR,0.4116,0.2842,0.5988,0.6809,0.7470,-0.0186,1.0000,0.7457,0.6234,0.7403,0.3284
PSP,0.5087,0.4838,0.7865,0.9087,0.8240,-0.2459,0.7457,1.0000,0.8246,0.9034,0.2287
QAI,0.6412,0.5305,0.7924,0.8370,0.7608,-0.0400,0.6234,0.8246,1.0000,0.8322,0.4028
SPY,0.4272,0.4909,0.7346,0.8740,0.7576,-0.2692,0.7403,0.9034,0.8322,1.0000,0.2033


In [333]:
corr[corr == 1] = np.nan
corr.unstack().sort_values(ascending=False)[:1]

PSP  EFA    0.908746
dtype: float64

In [334]:
corr[corr == 1] = np.nan
corr.unstack().sort_values(ascending=True)[:1]

IEF  DBC   -0.405431
dtype: float64

### The highest correlation pair is PSP/EFA
### Lowest is IEF/DBC

###  2.(b)How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?

In [335]:
table.loc[['TIP','IEF','BWX']]

,Mean,Vol,Sharpe Ratio
TIP,0.034967,0.047833,0.731032
IEF,0.025833,0.060077,0.429996
BWX,0.004653,0.078535,0.059248


### YES TIPS have done well compared to 7-10y UST and IG Sovereign Foreign Debt instruments. It has higher Sharpe Ratio, i.e., higher returns per unit of vol/sd.

### 2.(c) Based on the data, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

In [336]:
corr[-1:]

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
TIP,0.57696,0.073191,0.229728,0.200071,0.308617,0.68032,0.328398,0.228678,0.402777,0.203344,NaN


#### Yes, TIPS show low correlation to most of the asset, other than BWX and IEF with 57.7% and 68% correlation respectively.

## 3. The MV frontier.

## 3.(a) Compute and display the weights of the tangency portfolios: $\omega^t$.


In [337]:
def tangency_portfolio(df):
    N = df.shape[1]
    mu = df.mean() * 12 #mean
    sig = df.cov() * 12 #covariance
    sig_inv = np.linalg.inv(sig) #inverse of covariance
    
    scaling = np.ones(N)@sig_inv@mu
    
    om_t = (1/scaling) * sig_inv @ mu
    
    return om_t

omega_tangent = tangency_portfolio(ret)
omega_tangent = pd.DataFrame(omega_tangent, index=ret.columns, columns=['Weights'])
omega_tangent

,Weights
BWX,-1.335168
DBC,0.239151
EEM,0.339786
EFA,-0.117068
HYG,1.070489
IEF,2.457952
IYR,-0.307783
PSP,-0.513078
QAI,-3.955222
SPY,2.430623


## 3.(b) Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to $\omega^t$

In [338]:
# Calclulating mean, vol and sharpe ratio of tangency portfolio

def portfolio_stats(df, weights):
    N = df.shape[1]
    mu = df.mean() * 12 #mean
    sig = df.cov() * 12 #covariance
    
    mu_p = weights.T @ mu
    sig_p = np.sqrt(weights.T @ sig @ weights)
    sharpe_p = mu_p/sig_p
    
    print('Mean: ', mu_p, ',\nVol: ', sig_p, ',\nSharpe Ratio: ', sharpe_p)


In [339]:
portfolio_stats(ret, omega_tangent)

Mean:  Weights    0.342822
dtype: float64 ,
Vol:            Weights
Weights  0.175865 ,
Sharpe Ratio:            Weights
Weights  1.949342


## The allocation.
### 4.(a) Compute and display the weights of MV portfolios with target returns of $μ^p = .015$

In [340]:
mean_target = 0.015

In [341]:
#Calculate the minimum variance portfolio

def min_variance_portfolio(df):
    N = df.shape[1]
    mu = df.mean() * 12 #mean
    sig = df.cov() * 12 #covariance
    sig_inv = np.linalg.inv(sig) #inverse of covariance
    
    scaling = np.ones(N) @ sig_inv @ np.ones(N)
    omega_v = (1/scaling) * sig_inv @ np.ones(N)
    
    return omega_v

omega_variance = min_variance_portfolio(ret)
omega_variance = pd.DataFrame(omega_variance, index=ret.columns, columns=['Weights'])
omega_variance.T

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Weights,-0.124649,0.0056,-0.068993,0.034347,0.119633,0.095434,-0.011567,-0.109676,0.977931,-0.067959,0.149899


In [342]:
ret.mean()

BWX    0.000388
DBC    0.003237
EEM    0.006052
EFA    0.006760
HYG    0.005966
IEF    0.002153
IYR    0.012511
PSP    0.011106
QAI    0.001905
SPY    0.012524
TIP    0.002914
dtype: float64

In [343]:
delta = (mean_target - ret.mean().T @ omega_variance) / (ret.mean().T @ omega_tangent - ret.mean().T @ omega_variance)
delta

Weights    0.511554
dtype: float64

In [347]:
omega_p = omega_variance + delta * (omega_tangent - omega_variance)
omega_p

,Weights
BWX,-0.743896
DBC,0.125074
EEM,0.140120
EFA,-0.043110
HYG,0.606047
IEF,1.303991
IYR,-0.163098
PSP,-0.316038
QAI,-1.545644
SPY,1.210202


### 4.(b) What is the mean, volatility, and Sharpe ratio for $\omega^p$?

In [348]:
#omega_p = omega_p.squeeze()
portfolio_stats(ret, omega_p)

Mean:  Weights    0.18
dtype: float64 ,
Vol:            Weights
Weights  0.093406 ,
Sharpe Ratio:            Weights
Weights  1.927061



### 4.(c) Discuss the allocation. In which assets is the portfolio most long? And short?

- The portfolio is most long in IEF which are 7-10 year USTs. and biggest short position is on QAI the diversified asset.
- The portfolio also seems to have statistically relevant long positions in other TIP and HYG bonds along with a much higher allocation to SPY

In [349]:
omega_p.sort_values(by='Weights', ascending=False).T

,IEF,SPY,HYG,TIP,EEM,DBC,EFA,IYR,PSP,BWX,QAI
Weights,1.303991,1.210202,0.606047,0.426352,0.14012,0.125074,-0.04311,-0.163098,-0.316038,-0.743896,-1.545644


### 4.(d) Does this line up with which assets have the strongest Sharpe ratios?

No, it doesnt seem to follow the Sharpe ratio.

In [350]:
table.sort_values(by='Sharpe Ratio', ascending=False).T

,SPY,IYR,HYG,TIP,PSP,EFA,QAI,IEF,EEM,DBC,BWX
Mean,0.150293,0.150128,0.071588,0.034967,0.133272,0.081124,0.022862,0.025833,0.072621,0.038846,0.004653
Vol,0.144811,0.184407,0.089403,0.047833,0.221299,0.161885,0.048879,0.060077,0.191787,0.180186,0.078535
Sharpe Ratio,1.037857,0.814113,0.800730,0.731032,0.602227,0.501125,0.467723,0.429996,0.378655,0.215590,0.059248


## 5. Simple Portfolios

### 5.(a) Calculate the performance of the equally-weighted portfolio over the sample. Rescale the entire weighting vector to have target mean $μ^p = .015$. Report its mean, volatility, and Sharpe ratio.

In [351]:
weights_equal = np.ones(len(omega_tangent))/len(omega_tangent)
weights_equal

array([0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
       0.09090909, 0.09090909, 0.09090909, 0.09090909, 0.09090909,
       0.09090909])

In [353]:
weights_equal = weights_equal * (mean_target*12 / (ret_mean @ weights_equal))
weights_equal
portfolio_stats(ret, weights_equal)

Mean:  0.17999999999999997 ,
Vol:  0.25166093350987956 ,
Sharpe Ratio:  0.7152480819711082


## 5.(b) Calculate the performance of the “risk-parity” portfolio over the sample. Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample volatility estimate. $w^i = 1/\sigma_i$ This will give the weight vector, $\omega$, but you will need to rescale it to have a target mean of $μ^p = .015$.

In [355]:
weight_parity = 1/ret_vol
weight_parity = weight_parity * (mean_target*12 / (ret_mean @ weight_parity))
weight_parity

BWX    0.379571
DBC    0.165439
EEM    0.155431
EFA    0.184142
HYG    0.333430
IEF    0.496193
IYR    0.161652
PSP    0.134704
QAI    0.609866
SPY    0.205852
TIP    0.623205
dtype: float64

In [356]:
portfolio_stats(ret, weight_parity)

Mean:  0.17999999999999997 ,
Vol:  0.23560722729202044 ,
Sharpe Ratio:  0.763983355132401


### 5.(c) How does these compare to the MV portfolio from problem 2.4?

In [358]:
portfolio_stats(ret, omega_tangent)

Mean:  Weights    0.342822
dtype: float64 ,
Vol:            Weights
Weights  0.175865 ,
Sharpe Ratio:            Weights
Weights  1.949342


#### The sharpe ratio is lower compare to the MV portfolio. The Vols and Mean are individually lower compared to MV potfolio.
#### No short positions!

### 6. Assess how much the Sharpe Ratio goes down if we drop TIPS from the investment set, (and just have a 10-asset problem.) See how much it decreases the performance statistics in 2.4. And how much worse is the performance in 3.3?

### The sharpe ratio went down from 1.95 to around 1.91 after we dropped TIPS from the investment set

In [359]:
ret_tips = ret.drop(['TIP'], axis=1)
ret_tips.shape
omega_v_tips = min_variance_portfolio(ret_tips)
omega_t_tips = tangency_portfolio(ret_tips)
delta_tips = (mean_target - ret_tips.mean().T @ omega_v_tips) / (ret_tips.mean().T @ omega_t_tips - ret_tips.mean().T @ omega_v_tips)
omega_p_tips = omega_v_tips + delta_tips * (omega_t_tips - omega_v_tips)

portfolio_stats(ret_tips, omega_p_tips)


Mean:  0.18000000000000002 ,
Vol:  0.09408853907751885 ,
Sharpe Ratio:  1.913091666262342


### 7.Out-of-Sample Performance. Let’s divide the sample to both compute a portfolio and then check its performance out of sample.

### 7.(a) Using only data through the end of 2021, compute $\omega^p for μ^p $= .015, allocating to all 11 assets.

In [360]:
ret_2021 = pd.read_excel('multi_asset_etf_data.xlsx',2)
ret_2021.set_index('Date', inplace=True)
del ret_2021['SHV']

ret_2021.index = pd.to_datetime(ret_2021.index)
#remove the rows above 2022-01-01

ret_2021 = ret_2021[ret_2021.index < '2022-01-01']
ret_2021

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2009-04-30,0.008993,-0.001000,0.155582,0.115190,0.138460,-0.027452,0.296151,0.230202,0.022882,0.099346,-0.017952
2009-05-31,0.053672,0.162663,0.159400,0.131918,0.028555,-0.020773,0.022727,0.053892,0.027865,0.058454,0.019967
2009-06-30,0.005149,-0.026259,-0.022495,-0.014050,0.033516,-0.005572,-0.024863,0.045449,-0.003436,-0.000655,0.001982
2009-07-31,0.031284,0.018568,0.110146,0.100415,0.069191,0.008317,0.105799,0.143247,0.015326,0.074606,0.000879
2009-08-31,0.007628,-0.040365,-0.013136,0.045031,-0.016969,0.007635,0.131939,0.033413,-0.004151,0.036939,0.008413
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,-0.009051,-0.016410,0.015698,0.014467,0.006087,-0.003944,0.019396,0.014277,0.001244,0.029760,-0.001641
2021-09-30,-0.023686,0.052138,-0.038733,-0.032614,-0.003701,-0.015971,-0.056269,-0.083231,-0.013358,-0.046605,-0.007801
2021-10-31,-0.004493,0.057978,0.010718,0.031791,-0.003108,-0.004429,0.072761,0.094793,0.010705,0.070163,0.011095


In [365]:
omega_v_2021 = min_variance_portfolio(ret_2021)
omega_t_2021 = tangency_portfolio(ret_2021)
delta_2021 = (mean_target - ret_2021.mean().T @ omega_v_2021) / (ret_2021.mean().T @ omega_t_2021 - ret_2021.mean().T @ omega_v_2021)
omega_p_2021 = omega_v_2021 + delta_2021 * (omega_t_2021 - omega_v_2021)

pd.DataFrame(omega_p_2021, index=ret_2021.columns, columns=['Weights'])

,Weights
BWX,-0.260550
DBC,-0.013134
EEM,0.003842
EFA,-0.058326
HYG,0.665042
IEF,1.186294
IYR,-0.257773
PSP,-0.087996
QAI,-1.587112
SPY,1.109137


### 7.(b) Using those weights, calculate the portfolio’s Sharpe ratio within that sample, through the end of 2021.

In [366]:
portfolio_stats(ret_2021, omega_p_2021)

Mean:  0.18000000000000005 ,
Vol:  0.07798828578469066 ,
Sharpe Ratio:  2.308038934166887


### 7.(c)Again using those weights, (derived using data through 2021,) calculate the portfolio’s Sharpe ratio based on performance in 2022.

In [383]:
ret_2022 = pd.read_excel('multi_asset_etf_data.xlsx',2)
ret_2022.set_index('Date', inplace=True)
ret_2022.index = pd.to_datetime(ret_2022.index)
ret_2022 = ret_2022[ret_2022.index >= '2022-01-01']

omega_v_2022 = min_variance_portfolio(ret_2022)
omega_t_2022 = tangency_portfolio(ret_2022)
delta_2022 = (mean_target - ret_2022.mean().T @ omega_v_2022) / (ret_2022.mean().T @ omega_t_2022 - ret_2022.mean().T @ omega_v_2022)
omega_p_2022 = omega_v_2022 + delta_2022 * (omega_t_2022 - omega_v_2022)

pd.DataFrame(omega_p_2022, index=ret_2022.columns, columns=['Weights'])


,Weights
BWX,-0.697581
DBC,0.355494
EEM,0.165229
EFA,-1.483127
HYG,0.915300
IEF,1.527563
IYR,0.197113
PSP,0.246686
QAI,0.671317
SHV,0.391893


In [378]:
portfolio_stats(ret_2022, omega_p_2022)

Mean:  0.17999999999999985 ,
Vol:  2.7563081848147856e-09 ,
Sharpe Ratio:  65304743.85689757
